In [11]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
from utils import *
from cross_diffusion_process import *
import pickle
import classify_kernel_AD

In [12]:
K_X = torch.load('data/KernelData/K_X.pt')
K_y = torch.load('data/KernelData/K_y.pt')

with open('data/KernelData/Z.pkl', 'rb') as f:
    Z = pickle.load(f)
    
K_X.shape, K_y.shape

(torch.Size([5, 231, 231]), torch.Size([1, 231, 231]))

In [13]:
K_X = K_X[[0,1,3]]
K_X

tensor([[[0.0000, 0.0000, 0.1190,  ..., 0.0238, 0.0000, 0.0238],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.1190, 0.0000, 0.0000,  ..., 0.0714, 0.0000, 0.0000],
         ...,
         [0.0238, 0.0000, 0.0714,  ..., 0.0000, 0.0000, 0.1429],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0238, 0.0000, 0.0000,  ..., 0.1429, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.3333,  ..., 0.0000, 0.1667, 0.0714],
         [0.0000, 0.0000, 0.0238,  ..., 0.1429, 0.0000, 0.0238],
         [0.3333, 0.0238, 0.0000,  ..., 0.0000, 0.0476, 0.0952],
         ...,
         [0.0000, 0.1429, 0.0000,  ..., 0.0000, 0.0476, 0.2143],
         [0.1667, 0.0000, 0.0476,  ..., 0.0476, 0.0000, 0.2143],
         [0.0714, 0.0238, 0.0952,  ..., 0.2143, 0.2143, 0.0000]],

        [[0.0000, 0.7143, 0.2381,  ..., 0.0476, 0.0000, 0.1905],
         [0.7143, 0.0000, 0.2381,  ..., 0.0714, 0.0000, 0.1905],
         [0.2381, 0.2381, 0.0000,  ..., 0.1429, 0.0952, 0.

In [14]:
import numpy as np
from tensorly.decomposition import parafac
from sklearn.preprocessing import label_binarize
# from cross_diffusion_process import cross_diffusion_process
# from mkl_wrap import MKL_wrap
# from classify_kernel_AD import classify_kernel_AD

M, N, _ = K_X.shape



# Chuyển K_X thành 1 tensor mới với mỗi 1 layer là 1 modality
X_ten = np.zeros((N, N, M))
for ii in range(M):
    X_ten[:, :, ii] = K_X[ii, :, :]
    K_temp = X_ten[:, :, ii]
    np.fill_diagonal(K_temp, 1)
    X_ten[:, :, ii] = K_temp

X_ten = X_ten.astype(float)
X_ten.shape

# # Transform from array to tensor
# X_ten = tensor(X_ten)

(231, 231, 3)

In [15]:
X_ten

array([[[1.        , 1.        , 1.        ],
        [0.        , 0.        , 0.71428573],
        [0.11904762, 0.33333334, 0.23809524],
        ...,
        [0.02380952, 0.        , 0.04761905],
        [0.        , 0.16666667, 0.        ],
        [0.02380952, 0.07142857, 0.19047619]],

       [[0.        , 0.        , 0.71428573],
        [1.        , 1.        , 1.        ],
        [0.        , 0.02380952, 0.23809524],
        ...,
        [0.        , 0.14285715, 0.07142857],
        [0.        , 0.        , 0.        ],
        [0.        , 0.02380952, 0.19047619]],

       [[0.11904762, 0.33333334, 0.23809524],
        [0.        , 0.02380952, 0.23809524],
        [1.        , 1.        , 1.        ],
        ...,
        [0.07142857, 0.        , 0.14285715],
        [0.        , 0.04761905, 0.0952381 ],
        [0.        , 0.0952381 , 0.21428572]],

       ...,

       [[0.02380952, 0.        , 0.04761905],
        [0.        , 0.14285715, 0.07142857],
        [0.07142857, 0

In [16]:
# %pip install tensorly

# R là rank mong muốn cho phân rã CP
R = 2

# Thực hiện phân rã CP sử dụng parafac
weights, factors = parafac(X_ten, rank=R, tol=1e-9, n_iter_max=1000)

# factors là một list chứa các factor matrices của các mode trong CP decomposition
# errors là một list chứa các giá trị lỗi tái tạo ở mỗi vòng lặp

# Trích xuất các factor matrices từ list factors
factor_matrix_mode_0 = factors[0]
factor_matrix_mode_1 = factors[1]
# và cứ tiếp tục cho các mode khác

# Ví dụ: Hiển thị factor matrix của mode 0
print("Factor matrix của mode 0:")
print(factor_matrix_mode_0)

# Ví dụ: Hiển thị factor matrix của mode 1
print("Factor matrix của mode 1:")
print(factor_matrix_mode_1)


Factor matrix của mode 0:
[[ 5.26282822 -1.4192481 ]
 [ 4.2282386  -1.23197731]
 [ 3.91877676 -0.53116853]
 [ 5.0916417  -1.72545392]
 [ 5.15108369 -1.53290707]
 [ 3.65905805 -1.11693024]
 [ 5.84342047 -1.99054201]
 [ 5.12001685  0.13210457]
 [ 4.62010876 -0.93851261]
 [ 4.8535996  -0.8415714 ]
 [ 4.12594606 -1.4631743 ]
 [ 4.93801241 -1.31375113]
 [ 4.59525657 -1.14733223]
 [ 5.60692586 -1.81892821]
 [ 5.55007955 -1.88934138]
 [ 4.87772105 -0.81439352]
 [ 5.22675226 -1.12464935]
 [ 5.76195544 -0.75572914]
 [ 5.2571584  -1.62497791]
 [ 4.89726606 -1.53068909]
 [ 4.97118082 -0.9966144 ]
 [ 5.08692675 -1.39351058]
 [ 3.94393648 -1.20187422]
 [ 5.10056502 -1.50128677]
 [ 5.71330473 -1.93295135]
 [ 3.35312998 -0.36577729]
 [ 2.84595245 -0.41718939]
 [ 5.21084653 -1.65325056]
 [ 3.34487891 -0.45783365]
 [ 2.50889524 -0.29914486]
 [ 3.97081361 -0.49033283]
 [ 4.91495514 -0.13893875]
 [ 5.26345835 -1.38926054]
 [ 4.64912076 -1.22641597]
 [ 5.27570369 -1.86828166]
 [ 4.52635952 -0.53110702]
 [

In [17]:
factors[0].shape, factors[1].shape, factors[2].shape

((231, 2), (231, 2), (3, 2))

In [18]:
# Cross-diffusion process
kNN = 3
iter = 5
X_diffusion = cross_diffusion_process(X_ten, kNN, iter)
for ii in range(M):
    K_temp = X_diffusion[:, :, ii]
    np.fill_diagonal(K_temp, 1)
    X_diffusion[:, :, ii] = K_temp

## Simple MKL

In [26]:
factors[0].shape[0]

231

In [49]:
Result = factors
Y = np.squeeze(K_y[0, 0, :])
# Y = (np.array(Y, dtype=float) * 2) - 3
M = Result[2].shape[0]
N = Result[0].shape[0]
for ii in range(M):
    K_temp = Result[:, :, ii]
    np.fill_diagonal(K_temp, 1)
    Result[:, :, ii] = K_temp

TypeError: list indices must be integers or slices, not tuple

In [33]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from time import time

def MKL_wrap(K, Y, Z):
    C = 1
    verbose = 1

    options = {
        'algo': 'svmclass',
        'stopvariation': 0,
        'stopKKT': 0,
        'stopdualitygap': 1,
        'seuildiffsigma': 1e-2,
        'seuildiffconstraint': 0.1,
        'seuildualitygap': 0.01,
        'goldensearch_deltmax': 1e-1,
        'numericalprecision': 1e-8,
        'lambdareg': 1e-8,
        'firstbasevariable': 'first',
        'nbitermax': 500,
        'seuil': 0,
        'seuilitermax': 10,
        'miniter': 0,
        'verbosesvm': 0,
        'efficientkernel': 0
    }

    N = K.shape[1]
    M = K.shape[0]

    train_index = np.concatenate([Z[0], Z[1]], axis=0)  # train + val
    test_index = np.array(Z[2])

    K_train = K[train_index][:, train_index]
    y_train = Y[train_index]

    K_test = K[test_index][:, train_index]
    y_test = Y[test_index]

    # Training SVM
    start_time = time()
    clf = SVC(kernel='precomputed', C=C)
    clf.fit(K_train, y_train)
    end_time = time()
    timelasso = end_time - start_time

    # Making predictions
    y_pred_train = clf.predict(K_train)
    bc_train = np.mean(y_pred_train == y_train)

    K_test_combine = 0
    for i in range(len(clf.dual_coef_)):
        if i == 0:
            K_test_combine = clf.dual_coef_[i] * K_test[:, :, i]
        else:
            K_test_combine += clf.dual_coef_[i] * K_test[:, :, i]

    K_combine = 0
    for i in range(len(clf.dual_coef_)):   # dual_coef_ is the weight of each kernel
        if i == 0:
            K_combine = clf.dual_coef_[i] * K[:, :, i]
        else:
            K_combine += clf.dual_coef_[i] * K[:, :, i]

    y_pred_test = clf.predict(K_test_combine)
    bc_test = np.mean(y_pred_test == y_test)

    return clf.dual_coef_, K_combine, bc_test

In [38]:
train_index =  np.concatenate([Z[0], Z[1]], axis=0)
test_index = np.array(Z[2])
train_index, test_index

(array([ 47,  63, 116,  61,  94,  56,  39,  65, 102,  32,  21,  98,  12,
        117, 107,  13,  44,  78,  53,  46,  36, 103,  55,  28,  48, 105,
          0,  11,  72,  33,  68,  54,  22,  23,  50,  14,  86,  85,   5,
        111,  59,  90,  95,   1,  49,  91,  17, 114,  41,  27,  79,  62,
         58,  69,  20,  81,   4,  88,  74,  24,  57, 106,  29,  35, 110,
         97,  51,  70,  99,  89,  43,  64, 120,  67,  80, 115,  92,   3,
         87,  75,   6, 167, 128, 149, 138, 152, 171, 141, 229, 208, 157,
        142, 225, 172, 164, 129, 191, 217, 192, 134, 139, 201, 124, 197,
        189, 222, 216, 211, 182, 148, 174, 173, 136, 131, 190, 221, 219,
        202, 179, 133, 207, 214, 169, 203, 130, 155, 140, 166, 228, 210,
        199, 230, 158, 212, 187, 168, 177, 180, 204, 193, 161, 184, 163,
        223, 183, 176, 156,  10,  52,  71,  40,  76,  38, 100,   7,  26,
         83,  77,   8,  42, 109,  31,   2,  34,  37, 132, 121, 151, 188,
        220, 195, 165, 206, 227, 153, 160, 135, 186

In [1]:
K_train = X_ten[train_index][:, train_index]
y_train = Y[train_index]


NameError: name 'X_ten' is not defined

In [37]:
clf = SVC(kernel='precomputed', C=C)
clf.fit(K_train, y_train)



(184, 184, 3)

In [ ]:
# Making predictions
y_pred_train = clf.predict(K_train)
bc_train = np.mean(y_pred_train == y_train)

In [ ]:
K_combine = 0
for i in range(len(clf.dual_coef_)):   # dual_coef_ is the weight of each kernel
    if i == 0:
        K_combine = clf.dual_coef_[i] * K[:, :, i]
    else:
        K_combine += clf.dual_coef_[i] * K[:, :, i]

In [50]:
X_ten_reshaped = X_ten.reshape(N, N*M)
w1, K_combine, acc_test1 = MKL_wrap(np.array(X_ten_reshaped, dtype=float), Y, Z)


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed